# Pre-requisites  
- download dataset.zip,  
- unzip dataset,    
- mount drive,  

In [1]:
# Package installations
from IPython.utils import io
with io.capture_output() as captured:
    !pip install gdown
    !pip install torch==1.3.1+cu100 torchvision==0.4.2+cu100 -f https://download.pytorch.org/whl/torch_stable.html
    !pip3 install pillow==6.1

    # Install neptune.ai client for monitoring the training process
    !pip install neptune-client
    # !pip install psutil
    !pip install neptune-client neptune-tensorboard
    !pip install neptune-contrib
    !pip install pillow==4.1.1

In [ ]:
# Download VOC2012
# !gdown https://drive.google.com/uc?id=1PDTEuTnWJZNWogxYdqYGOlEZHK8dYET9

# Download custom Kvasir-v2
# ! gdown https://drive.google.com/uc?id=1WG5F7VVQe6mNupR1LLirkG_Apx6Bup5J

In [ ]:
# with io.capture_output() as captured:
#   # !unzip -q VOC2012.zip
#   !unzip -q Kvasir-v2.zip
#   # !rm -rf VOC2012.zip
#   !rm -rf Kvasir-v2.zip

In [1]:
import neptune
from neptunecontrib.monitoring.keras import NeptuneMonitor
from IPython.utils import io
# connect run to project
NEPTUNE_TOKEN = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhMGUxMmQ1NC00ZDU4LTQ4ZGYtOWJjOC0xYTJkYjJmYmJiZDMifQ=='
run = neptune.init(project_qualified_name='ch.kalavritinos/SEAM', api_token=NEPTUNE_TOKEN)

In [3]:
!git clone https://github.com/YudeWang/SEAM.git

Cloning into 'SEAM'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 50 (delta 11), reused 30 (delta 4), pack-reused 0
Unpacking objects: 100% (50/50), done.


In [4]:
%cd SEAM

/content/SEAM


In [2]:
with io.capture_output() as captured:
  from google.colab import drive 
  drive.mount('/content/drive')
%cd /content/drive/MyDrive/MAI/thesis/source

/content/drive/MyDrive/MAI/thesis/source


In [7]:
%cd /content/SEAM

/content/SEAM


In [ ]:
python infer_SEAM.py --weights $SEAM_weights --infer_list [voc12/val.txt | voc12/train.txt | voc12/train_aug.txt] --out_cam $your_cam_dir --out_crf $your_crf_dir

# SEAM infer
Imports

Change directory to SEAM




In [3]:
%cd SEAM/

/content/drive/MyDrive/MAI/thesis/source/SEAM


In [6]:
import sys
sys.path.append('/content/drive/MyDrive/MAI/thesis/source/SEAM')
import numpy as np
import torch
import random
import cv2
import os
from torch.utils.data import DataLoader
from torchvision import transforms
import voc12.data
import kvasirv2.data
from utils import pyutils, imutils, torchutils#, visualization
import argparse
import importlib
import torch.nn.functional as F

# PARAMETERS

In [ ]:
root_dir = 'OAA'
data_root = '/content/drive/MyDrive/MAI/thesis/source/kvasir-dataset-v2/'   # dataset directory
train_list = 'kvasirv2/train.txt'           # list of train images
test_list = 'kvasirv2/val.txt'              # list of val images
snapshot_dir = 'checkpoints/train/exp3/'    # where to save models
att_dir = './results_kvasir/exp3/attention/'   # where to save attentions
dataset = 'kvasir-v2'                       # dataset used


epoch = 14
lr = 0.001
batch_size = 32
input_size = 256
disp_interval = 100
num_classes = 20
num_workers = 8
weight_decay = 0.0005
decay_points = '5,10'

crop_size = 224         # for cropping images
threshold = 0.6         # for probabilities
disp_interval = 100     # display interval
resume  ='True'        # resume training 
global_counter = 0      
current_epoch = 0       # number of current epoch for resuming

# data.py  
for Kvasir-v2

In [ ]:
IMG_FOLDER_NAME = "polyps/"
ANNOT_FOLDER_NAME = "polyps/masks"

CAT_LIST = ['dyed-lifted-polyp',
            'dyed-resection-margins',
            'esophagitis',
            'normal-cecum',
            'normal-pylorus',
            'normal-z-line', 
            'polyps',
            'ulcerative-colitis']

CAT_NAME_TO_NUM = dict(zip(CAT_LIST,range(len(CAT_LIST))))


def load_image_label_list_from_npy(img_name_list):
    cls_labels_dict = np.load('kvasirv2/cls_labels.npy', allow_pickle=True).item()
    # print(cls_labels_dict)
    return [cls_labels_dict[img_name] for img_name in img_name_list]

def get_img_path(img_name, dataset_root):
    return os.path.join(dataset_root, img_name)

def load_img_name_list(dataset_path):
    img_gt_name_list = open(dataset_path).read().splitlines()
    img_name_list = [img_gt_name.split(' ')[0][-40:-4] for img_gt_name in img_gt_name_list]
    folder_paths_list = [img_gt_name.split(' ')[0] for img_gt_name in img_gt_name_list]
    return img_name_list, folder_paths_list

def load_label_list(dataset_path):
    zero_array = np.zeros(8, dtype=np.float32)
    list_of_arrays = []
    for i in range(8):
        z = np.zeros(8, dtype=np.float32)
        z[i]=1.0
        list_of_arrays.append(z)
    #print(list_of_arrays)
    img_name_list = open(dataset_path).read().splitlines()
    label_list = [list_of_arrays[int(img_name[-1:])-1] for img_name in img_name_list]
    return label_list

class KvasirImageDataset(Dataset):
    def __init__(self, img_name_list_path, dataset_root, transform=None):
        self.img_name_list, self.folder_paths_list = load_img_name_list(img_name_list_path)
        self.dataset_root = dataset_root
        self.transform = transform

    def __len__(self):
        return len(self.img_name_list)

    def __getitem__(self, idx):
        name = self.img_name_list[idx]
        path = self.folder_paths_list[idx]
        #print(get_img_path(path, self.dataset_root))
        img = PIL.Image.open(get_img_path(path, self.dataset_root)).convert("RGB")
        # img = torch.from_numpy(np.array(img))

        if self.transform:
            img = self.transform(img)
        return name, img

class KvasirClsDataset(KvasirImageDataset):           # inherit init from previous class
    def __init__(self, img_name_list_path, dataset_root, transform=None):
        super().__init__(img_name_list_path, dataset_root, transform)
        self.label_list = load_image_label_list_from_npy(self.img_name_list)        # get list from .npy file
        #self.label_list = load_image_label_list_from_xml(self.img_name_list, self.dataset_root)

    def __getitem__(self, idx):
        name, img = super().__getitem__(idx)
        label = torch.from_numpy(self.label_list[idx])
        return name, img, label

class KvasirClsDatasetMSF(KvasirClsDataset):
    def __init__(self, img_name_list_path, data_root, scales, inter_transform=None, unit=1):
        super().__init__(img_name_list_path, data_root, transform=None)
        self.scales = scales
        self.unit = unit
        self.inter_transform = inter_transform

    def __getitem__(self, idx):
        name, img, label = super().__getitem__(idx)
        rounded_size = (int(round(img.size[0]/self.unit)*self.unit), int(round(img.size[1]/self.unit)*self.unit))

        ms_img_list = []
        for s in self.scales:
            target_size = (round(rounded_size[0]*s),
                           round(rounded_size[1]*s))
            s_img = img.resize(target_size, resample=PIL.Image.CUBIC)
            ms_img_list.append(s_img)

        if self.inter_transform:
            for i in range(len(ms_img_list)):
                ms_img_list[i] = self.inter_transform(ms_img_list[i])

        msf_img_list = []
        for i in range(len(ms_img_list)):
            msf_img_list.append(ms_img_list[i])
            msf_img_list.append(np.flip(ms_img_list[i], -1).copy())
        return name, msf_img_list, label

class KvasirClsDatasetMS(KvasirClsDataset):
    def __init__(self, img_name_list_path, dataset_root, scales, inter_transform=None, unit=1):
        super().__init__(img_name_list_path, dataset_root, transform=None)
        self.scales = scales
        self.unit = unit
        self.inter_transform = inter_transform

    def __getitem__(self, idx):
        name, img, label = super().__getitem__(idx)
        rounded_size = (int(round(img.size[0]/self.unit)*self.unit), int(round(img.size[1]/self.unit)*self.unit))
        ms_img_list = []
        for s in self.scales:
            target_size = (round(rounded_size[0]*s),
                           round(rounded_size[1]*s))
            s_img = img.resize(target_size, resample=PIL.Image.CUBIC)
            ms_img_list.append(s_img)

        if self.inter_transform:
            for i in range(len(ms_img_list)):
                ms_img_list[i] = self.inter_transform(ms_img_list[i])

        return name, ms_img_list, label

# Losses

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        return 1 - dice

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        return Dice_BCE

class IoULoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(IoULoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection
        IoU = (intersection + smooth)/(union + smooth)

        return -IoU

class IoUBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(IoUBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection
        IoU = - (intersection + smooth)/(union + smooth)

        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        IoU_BCE = BCE + IoU

        return IoU_BCE

# train.py

In [ ]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):     # save points during training
    """Function for saving checkpoints"""
    savepath = os.path.join(snapshot_dir, filename)
    torch.save(state, savepath)
    if is_best:                                                         # if current epoch is best, then save checkpoint
        shutil.copyfile(savepath, os.path.join(snapshot_dir, 'model_best.pth.tar'))

def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch']

In [ ]:
def adaptive_min_pooling_loss(x):
    # This loss does not affect the highest performance, but change the optimial background score (alpha)
    n,c,h,w = x.size()
    k = h*w//4
    x = torch.max(x, dim=1)[0]
    y = torch.topk(x.view(n,-1), k=k, dim=-1, largest=False)[0]
    y = F.relu(y, inplace=False)
    loss = torch.sum(y)/(k*n)
    return loss

def max_onehot(x):
    n,c,h,w = x.size()
    x_max = torch.max(x[:,1:,:,:], dim=1, keepdim=True)[0]
    x[:,1:,:,:][x[:,1:,:,:] != x_max] = 0
    return x

In [ ]:
class VOCDataset(Dataset):
    def __init__(self, datalist_file, root_dir, num_classes=20, transform=None, test=False):
        self.root_dir = root_dir
        self.testing = test
        self.datalist_file =  datalist_file
        self.transform = transform
        self.num_classes = num_classes
        self.image_list, self.label_list = self.read_labeled_image_list(self.root_dir, self.datalist_file)

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name =  self.image_list[idx]
        image = PIL.Image.open(img_name).convert('RGB')
        image_arr = np.array(image)
        # print(image_arr.shape)
        
        if self.transform is not None:
            image = self.transform(image)
        # print("tranformed img:",np.array(image).shape)
        if self.testing:
            return img_name, image, self.label_list[idx]
        
        return image, self.label_list[idx]

    def read_labeled_image_list(self, data_dir, data_list):
        # print(data_dir)
        with open(data_list, 'r') as f:
            lines = f.readlines()
        img_name_list = []
        img_labels = []
        for line in lines:
            fields = line.strip().split()
            image = fields[0] + '.jpg'
            labels = np.zeros((self.num_classes,), dtype=np.float32)
            for i in range(len(fields)-1):
                index = int(fields[i+1])
                labels[index] = 1.
            img_name_list.append(os.path.join(data_dir, image))
            img_labels.append(labels)
        return img_name_list, img_labels#, np.array(img_labels, dtype=np.float32)

In [ ]:
def worker_init_fn(worker_id):
        np.random.seed(1 + worker_id)

def train(root_dir = 'OAA',
          # data_root = '/content/drive/MyDrive/MAI/thesis/source/kvasir-dataset-v2/',
          data_root = '/content/drive/MyDrive/MAI/thesis/source/VOCdevkit/',
          # train_list = 'kvasirv2/train.txt',
          train_list = 'voc12/train_oaa.txt',
          # val_list = 'kvasirv2/val.txt',              # list of val images
          val_list = 'voc12/val_oaa.txt',              # list of val images
          snapshot_dir = 'checkpoints/train/exp4/',    # where to save models
          # att_dir = './results_kvasir/exp3/attention/',   # where to save attentions
          att_dir = './results_voc/exp4/attention/',   # where to save attentions
          dataset = 'voc',                       # dataset used
          epoch = 14,
          lr = 0.001,
          batch_size = 8,
          input_size = 256,
          disp_interval = 100,
          num_classes = 20,
          num_workers = 8,
          weight_decay = 5e-4,
          decay_points = '5,10',
          crop_size = 448,         # for cropping images
          threshold = 0.6,         # for probabilities
          resume  = 'True',        # resume training 
          global_step = 0,      
          current_epoch = 0,
          network = "network.resnet38_SEAM",
          session_name = "resnet38_SEAM",
          weights = "",
          tblog_dir = "./tblog"):
    if not os.path.exists(snapshot_dir):
        os.makedirs(snapshot_dir)

    model = getattr(importlib.import_module(network), 'Net')() # init model
    
    # train_dataset = voc12.data.VOC12ClsDataset(train_list, voc12_root=data_root,
    #                                            transform=transforms.Compose([
    #                     imutils.RandomResizeLong(448, 768),
    #                     transforms.RandomHorizontalFlip(),
    #                     transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    #                     np.asarray,
    #                     model.normalize,
    #                     imutils.RandomCrop(crop_size),
    #                     imutils.HWC_to_CHW,
    #                     torch.from_numpy
    #                 ]))  # get class for train dataset
    # print('before making dataset...')

    train_dataset = kvasirv2.data.KvasirClsDataset(train_list, dataset_root=data_root,
                                               transform=transforms.Compose([
                        imutils.RandomResizeLong(448, 768),
                        transforms.RandomHorizontalFlip(),
                        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
                        np.asarray,
                        model.normalize,
                        imutils.RandomCrop(crop_size),
                        imutils.HWC_to_CHW,
                        torch.from_numpy
                    ]))
    # print('after')      # print type of dataset
    
    # Load train data uing default DataLoader
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size,
                                   shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=True,
                                   worker_init_fn=worker_init_fn)
    max_step = len(train_dataset) // batch_size * epoch             # calc step

    # print('max_step done')
    param_groups = model.get_parameter_groups()
    optimizer = torchutils.PolyOptimizer([              # use polyoptimizer 
        {'params': param_groups[0], 'lr': lr, 'weight_decay': weight_decay},
        {'params': param_groups[1], 'lr': 2*lr, 'weight_decay': 0},
        {'params': param_groups[2], 'lr': 10*lr, 'weight_decay': weight_decay},
        {'params': param_groups[3], 'lr': 20*lr, 'weight_decay': 0}
    ], lr=lr, weight_decay=weight_decay, max_step=max_step)

    if resume:
      model, optimizer, current_epoch = load_ckp(snapshot_dir+'voc_epoch_1.pth', model, optimizer)
      print('weights loaded!')

    if weights[-7:] == '.params':              # if file ends with .params extension import resnet38d
        # print('yes .params')
        import network.resnet38d
        # print(" resnet38d imported")
        assert 'resnet38' in network
        weights_dict = network.resnet38d.convert_mxnet_to_torch(weights)
        # print('weights loaded')
    else:
        weights_dict = torch.load(weights)
    # print('weights loaded')
    model.load_state_dict(weights_dict, strict=False)           # load state dict
    # print('state dict loaded')
    model = torch.nn.DataParallel(model).cuda()
    # print('parallel model ok')
    model.train()
    # print('train ok')

    avg_meter = pyutils.AverageMeter('loss', 'loss_cls', 'loss_er', 'loss_ecr')         # final loss = l_cls + l_er + l_ecr
    timer = pyutils.Timer("Session started: ")

    best_val_loss = float('inf')
    print('Training started ...')

    PARAMS = {'dataset':dataset,
                'network':'resnet38',
                'epoch_nr': epoch,
                'batch_size': batch_size,
                'optimizer': 'SGD',
                'lr': lr
          }
    neptune.create_experiment('voc12_train', params=PARAMS)

    while current_epoch < epoch:
        for iter, pack in enumerate(train_data_loader):
            scale_factor = 0.3
            img1 = pack[1]
            img2 = F.interpolate(img1,scale_factor=scale_factor,mode='bilinear',align_corners=True) 
            N,C,H,W = img1.size()       # get dims
            #print(N, C, H, W)
            label = pack[2]             # get label
            #print(label)
            bg_score = torch.ones((N,1))
            label = torch.cat((bg_score, label), dim=1)
            label = label.cuda(non_blocking=True).unsqueeze(2).unsqueeze(3)
            #print(label)
            cam1, cam_rv1 = model(img1)
            #print(type(cam1), type(cam_rv1))
            #print(cam_rv1)

            label1 = F.adaptive_avg_pool2d(cam1, (1,1))
            loss_rvmin1 = adaptive_min_pooling_loss((cam_rv1*label)[:,1:,:,:])
            cam1 = F.interpolate(visualization.max_norm(cam1),scale_factor=scale_factor,mode='bilinear',align_corners=True)*label
            cam_rv1 = F.interpolate(visualization.max_norm(cam_rv1),scale_factor=scale_factor,mode='bilinear',align_corners=True)*label

            cam2, cam_rv2 = model(img2)
            label2 = F.adaptive_avg_pool2d(cam2, (1,1))
            loss_rvmin2 = adaptive_min_pooling_loss((cam_rv2*label)[:,1:,:,:])
            cam2 = visualization.max_norm(cam2)*label
            cam_rv2 = visualization.max_norm(cam_rv2)*label

            loss_cls1 = F.multilabel_soft_margin_loss(label1[:,1:,:,:], label[:,1:,:,:])
            loss_cls2 = F.multilabel_soft_margin_loss(label2[:,1:,:,:], label[:,1:,:,:])

            ns,cs,hs,ws = cam2.size()
            loss_er = torch.mean(torch.abs(cam1[:,1:,:,:]-cam2[:,1:,:,:]))

            #loss_er = torch.mean(torch.pow(cam1[:,1:,:,:]-cam2[:,1:,:,:], 2))
            cam1[:,0,:,:] = 1-torch.max(cam1[:,1:,:,:],dim=1)[0]
            cam2[:,0,:,:] = 1-torch.max(cam2[:,1:,:,:],dim=1)[0]
#            with torch.no_grad():
#                eq_mask = (torch.max(torch.abs(cam1-cam2),dim=1,keepdim=True)[0]<0.7).float()
            tensor_ecr1 = torch.abs(max_onehot(cam2.detach()) - cam_rv1)#*eq_mask
            tensor_ecr2 = torch.abs(max_onehot(cam1.detach()) - cam_rv2)#*eq_mask
            loss_ecr1 = torch.mean(torch.topk(tensor_ecr1.view(ns,-1), k=(int)(2*hs*ws*0.2), dim=-1)[0])
            loss_ecr2 = torch.mean(torch.topk(tensor_ecr2.view(ns,-1), k=(int)(2*hs*ws*0.2), dim=-1)[0])
            loss_ecr = loss_ecr1 + loss_ecr2

            loss_cls = (loss_cls1 + loss_cls2)/2 + (loss_rvmin1 + loss_rvmin2)/2 
            loss = loss_cls + loss_er + loss_ecr

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            avg_meter.add({'loss': loss.item(), 'loss_cls': loss_cls.item(), 'loss_er': loss_er.item(), 'loss_ecr': loss_ecr.item()})

            if (optimizer.global_step - 1) % 50 == 0:
                timer.update_progress(optimizer.global_step / max_step)

                print('Iter:%5d/%5d' % (optimizer.global_step-1, max_step),
                      'loss:%.4f %.4f %.4f %.4f' % avg_meter.get('loss', 'loss_cls', 'loss_er', 'loss_ecr'),
                      'imps:%.1f' % ((iter+1) * batch_size / timer.get_stage_elapsed()),
                      'Fin:%s' % (timer.str_est_finish()),
                      'lr: %.4f' % (optimizer.param_groups[0]['lr']), flush=True)
                neptune.log_metric('train_loss', loss.item())
                neptune.log_metric('train_loss_cls', loss_cls.item())
                neptune.log_metric('train_loss_er', loss_er.item())
                neptune.log_metric('train_loss_ecr', loss_ecr.item())
                neptune.log_metric('lr', optimizer.param_groups[0]['lr'])

                avg_meter.pop()

                # Visualization for training process
                img_8 = img1[0].numpy().transpose((1,2,0))
                img_8 = np.ascontiguousarray(img_8)
                mean = (0.485, 0.456, 0.406)
                std = (0.229, 0.224, 0.225)
                img_8[:,:,0] = (img_8[:,:,0]*std[0] + mean[0])*255
                img_8[:,:,1] = (img_8[:,:,1]*std[1] + mean[1])*255
                img_8[:,:,2] = (img_8[:,:,2]*std[2] + mean[2])*255
                img_8[img_8 > 255] = 255
                img_8[img_8 < 0] = 0
                img_8 = img_8.astype(np.uint8)

                input_img = img_8.transpose((2,0,1))
                neptune.log_image(name = 'input img', filename = input_img)

                h = H//4; w = W//4
                p1 = F.interpolate(cam1,(h,w),mode='bilinear')[0].detach().cpu().numpy()
                p2 = F.interpolate(cam2,(h,w),mode='bilinear')[0].detach().cpu().numpy()
                p_rv1 = F.interpolate(cam_rv1,(h,w),mode='bilinear')[0].detach().cpu().numpy()
                p_rv2 = F.interpolate(cam_rv2,(h,w),mode='bilinear')[0].detach().cpu().numpy()

                image = cv2.resize(img_8, (w,h), interpolation=cv2.INTER_CUBIC).transpose((2,0,1))
                CLS1, CAM1, _, _ = visualization.generate_vis(p1, None, image, func_label2color=visualization.VOClabel2colormap, threshold=None, norm=False)
                CLS2, CAM2, _, _ = visualization.generate_vis(p2, None, image, func_label2color=visualization.VOClabel2colormap, threshold=None, norm=False)
                CLS_RV1, CAM_RV1, _, _ = visualization.generate_vis(p_rv1, None, image, func_label2color=visualization.VOClabel2colormap, threshold=None, norm=False)
                CLS_RV2, CAM_RV2, _, _ = visualization.generate_vis(p_rv2, None, image, func_label2color=visualization.VOClabel2colormap, threshold=None, norm=False)
                #MASK = eq_mask[0].detach().cpu().numpy().astype(np.uint8)*255
                loss_dict = {'loss':loss.item(), 
                             'loss_cls':loss_cls.item(),
                             'loss_er':loss_er.item(),
                             'loss_ecr':loss_ecr.item()}
                itr = optimizer.global_step - 1
                tblogger.add_scalars('loss', loss_dict, itr)
                tblogger.add_scalar('lr', optimizer.param_groups[0]['lr'], itr)
                tblogger.add_image('Image', input_img, itr)
                #tblogger.add_image('Mask', MASK, itr)
                tblogger.add_image('CLS1', CLS1, itr)
                tblogger.add_image('CLS2', CLS2, itr)
                tblogger.add_image('CLS_RV1', CLS_RV1, itr)
                tblogger.add_image('CLS_RV2', CLS_RV2, itr)
                tblogger.add_images('CAM1', CAM1, itr)
                tblogger.add_images('CAM2', CAM2, itr)
                tblogger.add_images('CAM_RV1', CAM_RV1, itr)
                tblogger.add_images('CAM_RV2', CAM_RV2, itr)

        else:
            print('')
            timer.reset_stage()
            save_checkpoint(
                          {'epoch': current_epoch,
                            'state_dict':model.state_dict(),
                            'optimizer':optimizer.state_dict()
                          }, is_best=False,
                          filename='%s_epoch_%d.pth' %(dataset, current_epoch))

    # torch.save(model.module.state_dict(),'pretrained_model/' + session_name + '.pth')           # save final model


# Train the CAM network

In [ ]:
train()

Training started ...
https://app.neptune.ai/ch.kalavritinos/OAA/e/OAA-24
Epoch: [0][100/2117]	LR: 0.00100	train loss 0.2427 (0.3020)	
Epoch: [0][200/2117]	LR: 0.00100	train loss 0.2099 (0.2510)	
Epoch: [0][300/2117]	LR: 0.00100	train loss 0.1191 (0.2287)	
Epoch: [0][400/2117]	LR: 0.00100	train loss 0.1343 (0.2107)	
Epoch: [0][500/2117]	LR: 0.00100	train loss 0.1949 (0.1970)	
